In [1]:
import json

import numpy as np
import pandas as pd

import andes
import ams

from andes.thirdparty.npfunc import safe_div

In [2]:
andes.config_logger(stream_level=50)
ams.config_logger(stream_level=50)

In [3]:
case_path = "./../cases"
res_path = "./../results"
addfile = case_path + '/IL200_dyn_db2.xlsx'

# --- file loading ---
curve = pd.read_csv(case_path + '/CurveInterp.csv')

In [4]:
sp = ams.load(case_path + '/IL200_opf2.xlsx',
              setup=True, no_output=True,
              default_config=True)

In [5]:
# NOTE: 1) the maximum number of dispatch should follow: "D * Dispatch_interval <= 3600"
#       2) the maximum number of AGC should follow: "Dispatch_interval % AGC_interval == 0"

Dispatch_interval = 900  # seconds
AGC_interval = 15  # seconds

pq_idx = sp.PQ.idx.v
p0 = sp.PQ.p0.v.copy()
q0 = sp.PQ.q0.v.copy()

stg = sp.StaticGen.get_all_idxes()

stg_idxes = sp.StaticGen.find_idx(keys='gentype',
                                  values=['W2', 'PV', 'ES'],
                                  allow_all=True)

stg_w2t, stg_pv, stg_ess = stg_idxes

p0_w2t = sp.StaticGen.get(src='p0', attr='v', idx=stg_w2t)
p0_pv = sp.StaticGen.get(src='p0', attr='v', idx=stg_pv)

In [6]:
export_dict = {}

for hour in range(24):
    if hour % 4 == 0:
        print(f"Hour {hour}")
    for dispatch in range(int(3600 / Dispatch_interval)):
        r0 = hour * 3600 + dispatch * Dispatch_interval
        r1 = r0 + Dispatch_interval

        load = curve['Load'].iloc[r0:r1].values.mean()
        sp.PQ.set(src='p0', attr='v', idx=pq_idx,
                  value=load*p0)
        sp.PQ.set(src='q0', attr='v', idx=pq_idx,
                  value=load*q0)

        psum = sp.PQ.p0.v.sum()
        solar = curve['PV'].iloc[r0:r1].values.mean()
        wind = curve['Wind'].iloc[r0:r1].values.mean()

        wind_sum = wind * p0_w2t.sum()
        solar_sum = solar * p0_pv.sum()
        # NOTE: discard wind and solar if they exceed the total load
        if wind_sum + solar_sum > psum:
            dgen = wind_sum + solar_sum - psum
            dwind = dgen / (wind_sum + solar_sum) * wind_sum
            dsolar = dgen / (wind_sum + solar_sum) * solar_sum
            wind = safe_div(wind_sum - 1.05*dwind, wind_sum)
            solar = safe_div(solar_sum - 1.05*dsolar, solar_sum)

        sp.StaticGen.set(src='p0', attr='v', idx=stg_w2t,
                         value=wind*p0_w2t)
        sp.StaticGen.set(src='p0', attr='v', idx=stg_pv,
                         value=solar*p0_pv)

        sp.StaticGen.set(src='p0', attr='v', idx=stg_w2t,
                         value=wind*p0_w2t)
        sp.StaticGen.set(src='p0', attr='v', idx=stg_pv,
                         value=solar*p0_pv)

        # NOTE: preserve 1% pmax as regulation capacity
        pmax = sp.StaticGen.get(src='pmax', attr='v', idx=stg)
        sp.StaticGen.set(src='pmax', attr='v', idx=stg,
                         value=0.99 * pmax)

        sp.ACOPF.config.update(verbose=0, out_all=0)

        sp.ACOPF.update()
        sp.ACOPF.run()

        if not sp.ACOPF.converged:
            print(f"Dispatch {dispatch} did not converge!")

        # Select variables to export
        export_dict[f"h{hour}d{dispatch}"] = {
            "converged": sp.ACOPF.converged,
            "load": load,
            "wind": wind,
            "solar": solar,
            "obj": sp.ACOPF.obj.v,
            "gen": sp.ACOPF.pg.get_all_idxes(),
            "pg": sp.ACOPF.pg.v.tolist(),
            "qg": sp.ACOPF.qg.v.tolist(),
            "bus": sp.ACOPF.vBus.get_all_idxes(),
            "vBus": sp.ACOPF.vBus.v.tolist(),
            "aBus": sp.ACOPF.aBus.v.tolist(),
        }

Hour 0
Hour 4
Hour 8
Hour 12
Hour 16
Hour 20


In [7]:
# Export to JSON file
with open(res_path + f"/opf.json", "w") as f:
    json.dump(export_dict, f, indent=2)